In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import alexnet
import pandas as pd
import numpy as np
from PIL import Image


class MNISTCSV(Dataset):
    def __init__(self, csv_file, is_train, transform=None):
        self.data = pd.read_csv(csv_file)
        self.is_train = is_train
        if is_train:
          self.labels = self.data['label'].values
          self.images = self.data.drop('label', axis=1).values.reshape(-1, 28, 28).astype(np.uint8)
        else:
          self.images = self.data.values.reshape(-1, 28, 28).astype(np.uint8)
        self.is_train = is_train
        self.transform = transform

    def __len__(self):
          return len(self.images)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        if self.transform:
            img = self.transform(img)
        if self.is_train:
          label = self.labels[idx]
          return img, label
        else:
          return img

# Step 1: Define transforms for AlexNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),                  # Resize to 224x224 for AlexNet
    transforms.Grayscale(num_output_channels=3),    # Convert 1-channel to 3-channel
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # ImageNet normalization
                         std=[0.229, 0.224, 0.225])
])

train_dir = 'train.csv'
test_dir = 'test.csv'

train_dataset = MNISTCSV(train_dir, True, transform=transform)
test_dataset = MNISTCSV(test_dir, False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


model = alexnet(pretrained=False)
model.classifier[6] = nn.Linear(4096, 10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100. * correct / total:.2f}%")




/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.3718, Accuracy: 87.73%
Epoch [2/10], Loss: 0.1122, Accuracy: 96.77%
Epoch [3/10], Loss: 0.0948, Accuracy: 97.33%
Epoch [4/10], Loss: 0.0856, Accuracy: 97.61%
Epoch [5/10], Loss: 0.0829, Accuracy: 97.73%
Epoch [6/10], Loss: 0.0727, Accuracy: 98.06%
Epoch [7/10], Loss: 0.0748, Accuracy: 98.01%
Epoch [8/10], Loss: 0.0596, Accuracy: 98.41%
Epoch [9/10], Loss: 0.0656, Accuracy: 98.21%
Epoch [10/10], Loss: 0.0583, Accuracy: 98.38%


In [16]:
predictions = []

for images in test_loader:
  outputs = model(images.to(device))
  _, predicted = outputs.max(1)
  predictions.extend(predicted.cpu().numpy())


submission_df = pd.DataFrame({'ImageId': range(1, len(predictions) + 1), 'Label': predictions})
submission_df.to_csv('submission.csv', index=False)

